# 발표 소단원 리스트

9-1,9-5,9-9,9-13,9-17,9-21,9-25

# 전체 요약자료

-


# 문항별 자료 
- (9-1) 함수에 wrapper 씌워서 새로운 함수 만들기
- (9-5) 속성으로 데코레이터 작성하기!
- (9-9) 데코레이터 클래스 작성하기
- (9-13) 메타클래스로 객체 생성 제약하기
- (9-17) 코딩 방식 제약하는 방법
- (9-21) 중복되는 기능 틀 정의하기
- (9-25) 바이트코드를 보기

### (9-1) Putting a Wrapper Around a Function
(sol) @wraps(func) 
- 함수를 인자로 받아서 함수를 출력함!!
- 다른 decorator도 비슷하게 사용함 (@staticmethod,@classmethod,@property)

In [1]:
import time
from functools import wraps

## wraper function
def timethis(func):
    '''
    Decorator that reports the execution time
    '''
    
    @wraps(func)
    def wrapper(*args,**kwargs):
        start = time.time()
        result = func(*args,**kwargs)
        end = time.time()
        print(func.__name__,end-start)
        return result
    
    return wrapper

In [2]:
@timethis
def countdown(n):
    while n > 0:
        n -= 1
        
## countdonw = timethis(countdown)

In [3]:
countdown(100000)

countdown 0.013035058975219727


In [5]:
countdown(10000000)

countdown 0.5860600471496582


In [9]:
countdown.__name__

'countdown'

In [8]:
countdown.__annotations__

{}

### (9-5) Defining a Decorator with user adjustable attributes
- accessor function + nonlocal variable

In [24]:
from functools import wraps,partial
import logging

# Utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj,func=None):
    if func is None:
        return partial(attach_wrapper,obj)
    setattr(obj,func.__name__,func)
    return func

def logged(level,name=None,message=None):
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args,**kwargs):
            log.log(level,logmsg)
            return func(*args,**kwargs)
        
        ## Attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
                
        @attach_wrapper(wrapper) ## wrapper에 한 번 더 감쌈!!
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
            
        return wrapper
    return decorate

# Example
@logged(logging.DEBUG) ## decorate함수를 아래 함수에 감싼다!!
def add(x,y):
    return x+y

In [32]:
add.__dict__

{'__wrapped__': <function __main__.add(x, y)>,
 'set_level': <function __main__.logged.<locals>.decorate.<locals>.set_level(newlevel)>,
 'set_message': <function __main__.logged.<locals>.decorate.<locals>.set_message(newmsg)>}

In [25]:
@logged(logging.CRITICAL,'example')
def span():
    print('Spam!')

In [26]:
import logging
logging.basicConfig(level=logging.DEBUG)
add(2,3)

DEBUG:__main__:add


5

In [27]:
add.set_message('Add called')
add(2,3)

DEBUG:__main__:Add called


5

In [30]:
add.set_level(logging.WARNING)
add(2,3)

5

### (9-9) Defining Decorators as Classes
- 클래스 내/외부에서 다 사용 가능한 decorator 정의

In [35]:
import types
from functools import wraps

class Profiled:
    def __init__(self,func):
        wraps(func)(self)
        self.ncalls = 0
        
    def __call__(self,*args,**kwargs):
        self.ncalls += 1
        return self.__wrapped__(*args,**kwargs)
    
    def __get__(self,instance,cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self,instance)

In [36]:
@Profiled
def add(x,y):
    return x+y

class Spam:
    @Profiled
    def bar(self,x):
        print(self,x)

In [37]:
add(2,3)

5

In [38]:
add.ncalls

1

In [39]:
s=Spam()

In [40]:
s.bar(1)

<__main__.Spam object at 0x000001BFD2F79CC0> 1


In [41]:
s.bar(2)

<__main__.Spam object at 0x000001BFD2F79CC0> 2


In [42]:
## class 변수 구현 가능!
Spam.bar.ncalls

2

### (9-13) Using a Metaclass to control Instance Creation
- singletons,caching 등 하기

In [ ]:
class Spam:
    def __init__(self,name):
        self.name = name
        
a = Spam('Guido')
b = Spam('Diana')

In [14]:
class NoInstances(type):
    def __call__(self,*args,**kwargs):
        raise TypeError("Can't instantiate directly")
        
class Spam(metaclass = NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')

In [15]:
Spam.grok(42)

Spam.grok


In [16]:
s = Spam()

TypeError: Can't instantiate directly

##### singleton

In [17]:
class Singleton(type):
    def __init__(self,*args,**kwargs):
        self.__instance = None
        super().__init__(*args,**kwargs)
        
    def __call__(self,*args,**kwargs):
        if self.__instance is None:
            self.__instance = super().__call__(*args,**kwargs)
            return self.__instance
        else:
            return self.__instance
        
class Spam(metaclass = Singleton):
    def __init__(self):
        print('Creating Spam')

In [18]:
a = Spam()
b = Spam()
a is b

Creating Spam


True

In [20]:
c = Spam()
a is c

True

In [21]:
import weakref

class Cached(type):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.__cache = weakref.WeakValueDictionary()
        
    def __call__(self,*args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj

class Spam(metaclass = Cached):
    def __init__(self,name):
        print('Creating Spam({!r})'.format(name))
        self.name = name
        
a = Spam('Guido')
b = Spam('Diana')
c = Spam('Guido')
a is b
        

Creating Spam('Guido')
Creating Spam('Diana')


False

In [22]:
a is c

True

In [ ]:
## metaclass 쓰면 깔끔한 코딩 가능
## 안 쓰고 singleton 만든 예제

class _Spam:
    def __init__(self):
        print('Creating Spam')
        
_spam_instance = None
def Spam():
    global _spam_instance
    
    if _spam_instance is None:
        _spam_instance = _Spam()
        
    return _spam_instance

### (9-17) Enforcing Coding Conventions in Classes
- (sol) __new__,__init__ 수정하기
- __new__는 클래스 객체 생성 전에 수행됨 / __init__는 클래스 객체 생성 후 수행됨

In [ ]:
class MyMeta(type):
    def __new__(self,clsname,bases,clsdict):
        return super().__new__(cls,clsname,bases,clsdict)
    
## init 쓰는 방법
class MyMeta(type):
    def __init__(self,clsname,bases,clsdict):
        super().__init__(clsname,bases,clsdict)
        

In [23]:
class NoMixedCaseMeta(type):
    def __new__(cls,clsname,bases,clsdict):
        for name in clsdict:
            if name.lower() != name:
                raise TypeError('Bad attribute name: '+name)
            return super().__new__(cls,clsname,bases,clsdict)
        
class Root(metaclass = NoMixedCaseMeta):
    pass

class A(Root):
    def foo_bar(self):
        pass
    
class B(Root):
    def fooBar(self): # TypeError
        pass

In [25]:
from inspect import signature
import logging

class MatchSignaturesMeta(type):
    def __init__(self,clsname,bases,clsdict):
        super().__init__(clsname,bases,clsdict)
        sup = super(self,self) # 특이함
        for name,value in clsdict.items():
            if name.startswith('_') or not callable(value):
                continue
            prev_dfn = getattr(sup,name,None)
            if prev_dfn:
                prev_sig = signature(prev_dfn)
                val_sig = signature(value)
                if prev_sig != val_sig:
                    logging.warning('Signature mismatch in %s. %s != %s',
                                   value.__qualname__,prev_sig,val_sig)
                    
# example
class Root(metaclass = MatchSignaturesMeta):
    pass

class A(Root):
    def foo(self,x,y):
        pass
    
    def spam(self,x,*,z):
        pass
    
# class with redefined methods
## subclass가 arguments name바꿨는지를 체크해줌!!
class B(A):
    def foo(self,a,b):
        pass
    def spam(self,x,z):
        pass

### (9-21) Avoiding Repetitive Property Methods
- (sol) 함수를 만들어 줌!

In [ ]:
class Person:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self,value):
        if not isinstance(value,str):
            raise TypeError('name must be a string')
        self.name = value
        
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self,value):
        if not isinstance(value,int):
            raise TypeError('age must be an int')
        self._age = value

In [ ]:
### 간단히 하는 방법
def typed_property(name,expected_type):
    storage_name = '_' + name
    
    @property
    def prop(self):
        return getattr(self,storage_name)
    
    @prop.setter
    def prop(self,value):
        if not isinstance(value,expected_type):
            raise TypeError('{} must be a {}'.format(name,expected_type))
        setattr(self,storage_name,value)
        
    return prop

class Person:
    name = typed_property('name',str)
    age = typed_property('age',int)
    def __init__(self,name,age):
        self.name = name
        self.age = age
        

In [ ]:
from functools import partial
String = partial(typed_property,expected_type = str)
Integer = partial(typed_property,expected_type = int)

class Person:
    name = String('name')
    age = Integer('age')
    def __init__(self,name,age):
        self.name = name
        self.age = age

### (9-25) Disassembling Python Byte Code
- (sol) dis 모듈을 사용하면 됨

In [1]:
def countdown(n):
    while n > 0:
        print('T-minus',n)
        n -= 1
    print('Blastoff!')

In [2]:
import dis
dis.dis(countdown)

  2           0 SETUP_LOOP              30 (to 32)
        >>    2 LOAD_FAST                0 (n)
              4 LOAD_CONST               1 (0)
              6 COMPARE_OP               4 (>)
              8 POP_JUMP_IF_FALSE       30

  3          10 LOAD_GLOBAL              0 (print)
             12 LOAD_CONST               2 ('T-minus')
             14 LOAD_FAST                0 (n)
             16 CALL_FUNCTION            2
             18 POP_TOP

  4          20 LOAD_FAST                0 (n)
             22 LOAD_CONST               3 (1)
             24 INPLACE_SUBTRACT
             26 STORE_FAST               0 (n)
             28 JUMP_ABSOLUTE            2
        >>   30 POP_BLOCK

  5     >>   32 LOAD_GLOBAL              0 (print)
             34 LOAD_CONST               4 ('Blastoff!')
             36 CALL_FUNCTION            1
             38 POP_TOP
             40 LOAD_CONST               0 (None)
             42 RETURN_VALUE


In [3]:
# dis 모듈이 interpret하는 것
countdown.__code__.co_code

b'x\x1e|\x00d\x01k\x04r\x1et\x00d\x02|\x00\x83\x02\x01\x00|\x00d\x038\x00}\x00q\x02W\x00t\x00d\x04\x83\x01\x01\x00d\x00S\x00'

In [8]:
c = countdown.__code__.co_code
import opcode
opcode.opname[c[0]]
opcode.opname[c[0]]

'SETUP_LOOP'

In [10]:
opcode.opname[c[2]]

'LOAD_FAST'

In [11]:
import opcode
def generate_opcodes(codebytes):
    extended_arg = 0 
    i = 0 
    n = len(codebytes)
    while i<n:
        op = codebytes[i]
        i+=1
        if op >= opcode.HAVE_ARGUMENT:
            oparg = codebytes[i] + codebytes[i+1]*256 + extended_arg
            extended_arg = 0
            i += 2
            if op == opcode.EXTENDED_ARG:
                extended_arg = oparg * 65536
                continue
                
        else:
            oparg = None
        yield (op,oparg)

In [12]:
for op,oparg in generate_opcodes(countdown.__code__.co_code):
    print(op,opcode.opname[op],oparg)

120 SETUP_LOOP 31774
0 <0> None
100 LOAD_CONST 27393
4 DUP_TOP None
114 POP_JUMP_IF_FALSE 29726
0 <0> None
100 LOAD_CONST 31746
0 <0> None
131 CALL_FUNCTION 258
0 <0> None
124 LOAD_FAST 25600
3 ROT_THREE None
56 INPLACE_SUBTRACT None
0 <0> None
125 STORE_FAST 28928
2 ROT_TWO None
87 POP_BLOCK None
0 <0> None
116 LOAD_GLOBAL 25600
4 DUP_TOP None
131 CALL_FUNCTION 257
0 <0> None
100 LOAD_CONST 21248
0 <0> None
